# Signal processing course 2018/2019-1 @ ELTE
# Assignment 3
## 10.08.2018

## Task 1

### Low-pass filter (LPF) and the Bode plot

A low-pass filter (LPF) is a filter that passes signals with a frequency lower than a selected cutoff frequency and attenuates signals with frequencies higher than the cutoff frequency. The exact frequency response of the filter depends on the filter design.[\[1\]](https://en.wikipedia.org/wiki/Low-pass_filter)
We can construct a wingle pole LPF, simplest way as shown in the first picture:

![Low-pass filter](img\low-pass_filter_RC.svg "First order low-pass filter")

This simple LPF circuit consists of a resistor in series with a load, and a capacitor in parallel with the load. The capacitor exhibits reactance, and blocks low-frequency signals, forcing them through the load instead. At higher frequencies the reactance drops, and the capacitor effectively functions as a short circuit. The combination of resistance and capacitance gives the time constant of the filter $\tau = RC$, where in SI, $R$ is measured in Ohms ($\Omega$) and $C$ is measured in Farads ($F$).[\[2\]](https://www.allaboutcircuits.com/textbook/direct-current/chpt-16/voltage-current-calculations/). The *break frequency*, also called the *turnover frequency* or *cutoff frequency* (in Hz), is determined by the time constant:

$$
f_{c} = \frac{1}{2 \pi} \omega_{c} = \frac{1}{2 \pi} \frac{1}{\tau} = \frac{1}{2 \pi} \frac{1}{RC}
$$

This circuit may be understood by considering the time the capacitor needs to charge or discharge through the resistor:

* At low frequencies, there is plenty of time for the capacitor to charge up to practically the same voltage as the input voltage.
* At high frequencies, the capacitor only has time to charge up a small amount before the input switches direction. The output goes up and down only a small fraction of the amount the input goes up and down. At double the frequency, there's only time for it to charge up half the amount.

Another way to understand this circuit is through the concept of reactance at a particular frequency:

* Since direct current (DC) cannot flow through the capacitor, DC input must flow out the path marked $V_{out}$ (analogous to removing the capacitor).
* Since alternating current (AC) flows very well through the capacitor, almost as well as it flows through solid wire, AC input flows out through the capacitor, effectively short circuiting to ground (analogous to replacing the capacitor with just a wire).

The capacitor is not an "on/off" object (like the block or pass fluidic explanation above). The capacitor variably acts between these two extremes. It is the *Bode plot* and frequency response that show this variability, which is as follows:

![Bode of LPF](img\lpf_bode.png "Bode plot of a first-order (one-pole) low-pass filter")

One can calculate the Bode plot for a filter, using its transfer function, as follows[\[3\]](https://slideplayer.com/slide/4225151/) (for an example we use the single pole LPF):

$$
H \left(j \omega \right) = \frac{1}{1 + j \omega RC} = \frac{1}{1 + j \tfrac{\omega}{w_{c}}}
$$

We need the absolute (real) values of this function, thus:

$$
\left| H \left(j \omega \right) \right| = \frac{1}{\sqrt{1 + \left( \tfrac{\omega}{w_{c}} \right)^{2}}}
$$

At the last step we convert this to decibels:

$$
\left| H \left(j \omega \right) \right|_{dB} = 20 \cdot \log_{10} \left| H \left(j \omega \right) \right| = 20 \cdot \log_{10} \left( \frac{1}{\sqrt{1 + \left( \tfrac{\omega}{w_{c}} \right)^{2}}} \right)
$$

### Nyquist plot

A Nyquist plot is a parametric plot of a frequency response used in signal processing. The most common use of Nyquist plots is for assessing the stability of a system with feedback. In Cartesian coordinates, the real part of the transfer function is plotted on the X axis. The imaginary part is plotted on the Y axis. The frequency is swept as a parameter, resulting in a plot per frequency.[\[4\]](https://en.wikipedia.org/wiki/Nyquist_stability_criterion)
Summed, one can construct the Nyquist plot, using the transfer function $H \left(j \omega \right)$ plotting the following function:

$$
H_{Im} \left( H_{Re} \right)
$$

It results in the following plot:

![Nyquist of LPF](img\lpf_nyquist.svg "Nyquist plot of a first-order (one-pole) low-pass filter")

In [5]:
# Resistance's magnitude in the low-pass filter
# Measured in Ohm (greek Omega)
LPF_R = 1e+5

# Capacitor's magnitude on the low-pass filter
# Measured Farads (F)
LPF_C = 1e-5

# Time constant for LPF with given characteristics
tau = LPF_R * LPF_C